In [1]:
import pickle
import torch
import pandas as pd
from torch_geometric.data import Data,DataLoader
from functions import *
from pytorch_util import *
from torch.optim import Adam
from torch.nn.utils import clip_grad_value_

In [2]:
with open('../Data/train_data3.pickle', 'rb') as handle:
    train_data = pickle.load(handle)
with open('../Data/val_data3.pickle', 'rb') as handle:
    val_data = pickle.load(handle)
with open('../Data/test_data3.pickle', 'rb') as handle:
    test_data = pickle.load(handle)

In [3]:
### parameters ###
batch_size = 32
dim = 64
edge_dim = 12
epochs = 5
clip = 2

### load dataset

In [4]:
train_list = [Data(**transform_xyz(d)) for d in train_data]
train_dl = DataLoader(train_list,batch_size,shuffle=True)

In [5]:
val_list = [Data(**d) for d in val_data]
valid_dl = DataLoader(val_list,batch_size,shuffle=False)

In [6]:
test_list = [Data(**d) for d in test_data]
test_dl = DataLoader(test_list,batch_size,shuffle=False)

In [7]:
class Net_int_couplingEdge(torch.nn.Module):
    # differ from Net in that edge_attr3 is used to determine Weight
    def __init__(self,dim=64,edge_dim=12):
        super(Net_int, self).__init__()
        self.lin_node = torch.nn.Linear(8, dim)
        self.lin_edge_attr = torch.nn.Linear(19, edge_dim)
        
        nn = Sequential(Linear(12, 128), ReLU(), Linear(128, dim * dim))
        self.conv = NNConv(dim, dim, nn, aggr='mean', root_weight=False)
        self.gru = GRU(dim, dim)
        self.lin_weight = Linear(8, dim*3, bias=False)
        self.lin_bias = Linear(8, 1, bias=False)
        self.norm = BatchNorm1d(dim*3)
        
    def forward(self, data,IsTrain=False):
        out = F.relu(self.lin_node(data.x))
        edge_attr = F.relu(self.lin_edge_attr(data.edge_attr))
        h = out.unsqueeze(0)

        for i in range(3):
            m = F.relu(self.conv(out, data.edge_index3, edge_attr3))
            out, h = self.gru(m.unsqueeze(0), h)
            out = out.squeeze(0)
        
        temp = out[data.edge_index3] # (2,N,d)
        yhat = torch.cat([temp.mean(0),temp[0]*temp[1],(temp[0]-temp[1])**2],1)
        yhat = self.norm(yhat)
        weight = self.lin_weight(data.edge_attr3)
        bias = self.lin_bias(data.edge_attr3)
        yhat = torch.sum(yhat * weight,1,keepdim=True) + bias
        yhat = yhat.squeeze(1)
        
        if IsTrain:
            k = torch.sum(data.edge_attr3,0)
            nonzeroIndex = torch.nonzero(k).squeeze(1)
            abs_ = torch.abs(data.y-yhat).unsqueeze(1)
            loss = torch.sum(torch.log(torch.sum(abs_ * data.edge_attr3[:,nonzeroIndex],0)/k[nonzeroIndex]))/nonzeroIndex.shape[0]
            return loss
        else:
            return yhat    

### build model and set up training

In [8]:
model = Net_int(dim=dim,edge_dim=edge_dim).to('cuda:0')

In [9]:
paras = trainable_parameter(model)
opt = Adam(paras,lr=1e-4)

In [10]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
    # Rotation transform
    train_list = [Data(**transform_xyz(d)) for d in train_data]
    train_dl = DataLoader(train_list,batch_size,shuffle=True)
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:1.334547744311859, val_loss:1.1262410016905549
epoch:1, train_loss:1.085568251842007, val_loss:1.0161523691609375
epoch:2, train_loss:0.980139206902426, val_loss:0.8919666585249778
epoch:3, train_loss:0.8210584249298051, val_loss:0.6724580381797929
epoch:4, train_loss:0.6619851374849643, val_loss:0.6177524382996763
Training completed in 159.37636041641235s


In [11]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
    # Rotation transform
    train_list = [Data(**transform_xyz(d)) for d in train_data]
    train_dl = DataLoader(train_list,batch_size,shuffle=True)
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:0.6244927106325519, val_loss:0.5894093563159307
epoch:1, train_loss:0.5891277922524346, val_loss:0.6086998008127905
epoch:2, train_loss:0.5574048575087122, val_loss:0.6908175423104539
epoch:3, train_loss:0.5329746077417182, val_loss:0.4999057015674746
epoch:4, train_loss:0.5119107034607571, val_loss:0.4701884283214553
Training completed in 160.0615644454956s


In [12]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
    # Rotation transform
    train_list = [Data(**transform_xyz(d)) for d in train_data]
    train_dl = DataLoader(train_list,batch_size,shuffle=True)
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:0.4927181729891009, val_loss:0.4658548873968614
epoch:1, train_loss:0.47712983868958253, val_loss:0.501036200258467
epoch:2, train_loss:0.4589469837962518, val_loss:0.4206883080431029
epoch:3, train_loss:0.44506942969672364, val_loss:0.4056434906764418
epoch:4, train_loss:0.4296151844262532, val_loss:0.46361770047846
Training completed in 158.88767647743225s


In [21]:
torch.save({'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, '../Model/gnn_int.tar')

### make submissions

In [22]:
model.eval()
yhat_list = []
with torch.no_grad():
    for data in test_dl:
        data = data.to('cuda:0')
        yhat_list.append(model(data,False))

yhat = torch.cat(yhat_list).cpu().detach().numpy()
submission = pd.read_csv('../Data/sample_submission.csv')
submission['scalar_coupling_constant'] = yhat
submission.to_csv('../Submission/sub_gnn_int.csv',index=False)